In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import lob_for_futures as lobFut
from lob_for_futures import *
import os
import matplotlib.pyplot as plt

from dateutil.parser import parse
from pandas.tseries.offsets import BDay
import pickle as pkl
import fnmatch
import pickle
import datetime as dt
from datetime import timedelta
import glob
import fathon
from fathon import fathonUtils as fu
import itertools
import pyinform as pyinf
import pingouin as pig
from scipy.stats import entropy
from scipy.spatial.distance import jensenshannon
from scipy.special import kl_div
import time
import matplotlib.ticker as mtick

dataFolder = lobFut.dataFolder
quotes = [f for f in os.listdir(dataFolder) if str('_quotes') in f]
trades = [f for f in os.listdir(dataFolder) if str('_trades') in f]

# symbolIdx = 0

/home/ak/anaconda3/envs/mmd-kernels/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.4.0, the latest is 0.5.0.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


In [2]:


symbols = [f.split("_")[0] for f in quotes]

from multiprocessing import Pool
import pickle

trades_cols = ['size', 'time', 'type', 'value']
experimentsDestination = '/media/ak/T7/MFDFA Experiments'
plt.style.use(os.path.join('/home/ak/.config/matplotlib', 'latexstyle.mplstyle'))
informationClockFolder = '/media/ak/T7/FuturesDataSemiProcessed'



In [20]:

def produce_info_clock_files(date):
    bar_returns = dict()
    bars_dicts = defaultdict(dict)
    informationClockFolder = '/media/ak/T7/FuturesDataSemiProcessed'

    df = testClass.load_and_format_data()[str(date)]

    input_dict = testClass.get_bars(df)
    calendar_bar_df = (testClass.get_concat_data(testClass._bars_dict)['calendar_bars'])
    tick_bar_df = testClass.get_concat_data(testClass._bars_dict)['tick_bars']
    volume_bar_df = (testClass.get_concat_data(testClass._bars_dict)['volume_bars'])
    usd_volume_bar_df = testClass.get_concat_data(testClass._bars_dict)['usd_volume_bars']
    vr = returns(volume_bar_df.micro_price_close).replace([np.inf, -np.inf], 0)  # volume
    tr = returns(tick_bar_df.micro_price_close).replace([np.inf, -np.inf], 0)  # tick
    dr = returns(usd_volume_bar_df.micro_price_close).dropna().replace([np.inf, -np.inf], 0)  # usd volume
    df_ret = returns(calendar_bar_df.micro_price_close).dropna().replace([np.inf, -np.inf], 0)  # calendar
    bar_returns[date] = {'tick': tr,
                         'volume': vr,
                         'dollar': dr,
                         'calendar': df_ret}
    bars_dicts[date]['tick'] = tick_bar_df
    bars_dicts[date]['volume'] = volume_bar_df
    bars_dicts[date]['calendar'] = calendar_bar_df
    bars_dicts[date]['dollar'] = usd_volume_bar_df
    print(bar_returns)
    symbolFolderToStore = os.path.join(informationClockFolder, symbol)

    #  store all values
    pickle_out_filename = os.path.join(symbolFolderToStore, str(date) + '.pkl')
    pickle_out_filename_returns = os.path.join(symbolFolderToStore, 'Returns_' + str(date) + '.pkl')

    pickle.dump(bars_dicts, open(pickle_out_filename, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
    pickle.dump(bar_returns, open(pickle_out_filename_returns, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)

    print('just saved: ', pickle_out_filename)

    return bar_returns, bars_dicts

In [4]:

symbolIdx = 9

symbol = sorted(symbols)[symbolIdx]
print(symbol)

symbolFolderToStore = os.path.join(informationClockFolder, symbol)
print(symbolFolderToStore)
number_of_workers = 5

quotesFileCh = os.path.join(dataFolder, quotes[symbolIdx])
tradesFileCh = os.path.join(dataFolder, trades[symbolIdx])


TU1
/media/ak/T7/FuturesDataSemiProcessed/TU1


In [5]:

# get common Dates
quotesDates = sorted([f.split(".csv")[0] for f in os.listdir(quotesFileCh)])
tradesDates = sorted([f.split(".csv")[0] for f in os.listdir(tradesFileCh)])
intersectionDates = list(set(quotesDates).intersection(tradesDates))

trades_cols = ['size', 'time', 'type', 'value']

# params for the clocks

calendar_resample_freq = "300S"
trade_volume_width = 100
ticks_width = 100
usd_volume_width = 100
dates_choice = intersectionDates

In [7]:


testClass = DataLoader(data_location=dataFolder,
                       symbol=symbol,
                       dates=dates_choice,
                       use_columns=trades_cols,
                       calendar_resample=calendar_resample_freq,
                       trade_volume_width=trade_volume_width,
                       ticks_width=ticks_width,
                       usd_volume_width=usd_volume_width)

# load data
input_dict = testClass.load_and_format_data()

2021-11-13 19:49:51,870 DataLoader  INFO Loading data for:20180725


/media/ak/WorkDrive/Data/TU1_Comdty_quotes/20180725.csv
/media/ak/WorkDrive/Data/TU1_Comdty_trades/20180725.csv


2021-11-13 19:49:52,395 DataLoader  INFO Loading data for:20180724


/media/ak/WorkDrive/Data/TU1_Comdty_quotes/20180724.csv
/media/ak/WorkDrive/Data/TU1_Comdty_trades/20180724.csv


2021-11-13 19:49:52,807 DataLoader  INFO Loading data for:20180517


...

/media/ak/WorkDrive/Data/TU1_Comdty_quotes/20180420.csv
/media/ak/WorkDrive/Data/TU1_Comdty_trades/20180420.csv


2021-11-13 19:50:33,110 DataLoader  INFO Loading data for:20180504


/media/ak/WorkDrive/Data/TU1_Comdty_quotes/20180504.csv
/media/ak/WorkDrive/Data/TU1_Comdty_trades/20180504.csv


2021-11-13 19:50:33,553 DataLoader  INFO Loading data for:20180913


/media/ak/WorkDrive/Data/TU1_Comdty_quotes/20180913.csv
/media/ak/WorkDrive/Data/TU1_Comdty_trades/20180913.csv


In [21]:
date = intersectionDates[0]
bar_returns, bars_dicts = produce_info_clock_files(date)

2021-11-13 20:15:33,490 DataLoader  INFO Loading data for:20180725


/media/ak/WorkDrive/Data/TU1_Comdty_quotes/20180725.csv
/media/ak/WorkDrive/Data/TU1_Comdty_trades/20180725.csv


2021-11-13 20:15:33,984 DataLoader  INFO Loading data for:20180724


/media/ak/WorkDrive/Data/TU1_Comdty_quotes/20180724.csv
/media/ak/WorkDrive/Data/TU1_Comdty_trades/20180724.csv


2021-11-13 20:15:34,395 DataLoader  INFO Loading data for:20180517


...

/media/ak/WorkDrive/Data/TU1_Comdty_quotes/20180504.csv
/media/ak/WorkDrive/Data/TU1_Comdty_trades/20180504.csv


2021-11-13 20:16:13,722 DataLoader  INFO Loading data for:20180913


/media/ak/WorkDrive/Data/TU1_Comdty_quotes/20180913.csv
/media/ak/WorkDrive/Data/TU1_Comdty_trades/20180913.csv


2021-11-13 20:16:14,395 DataLoader  INFO Calculating calendar clock
2021-11-13 20:16:15,695 DataLoader  INFO Calculating volume clock
2021-11-13 20:16:47,526 DataLoader  INFO Calculating tick clock
2021-11-13 20:16:48,842 DataLoader  INFO Calculating usd volume clock


{'20180725': {'tick': tick_bucket
2.0       2.666084e-06
3.0      -2.900328e-06
4.0      -3.096897e-06
5.0      -1.983080e-06
6.0      -7.613414e-07
              ...     
1618.0    1.331230e-05
1619.0   -5.017481e-06
1620.0    6.777258e-09
1621.0    1.599355e-05
1622.0    1.114729e-05
Length: 3243, dtype: float64, 'volume': volume_bucket
2.0        3.801245e-07
3.0        2.082894e-06
4.0        1.570772e-06
5.0       -1.872156e-07
7.0        7.931050e-07
               ...     
67900.0   -2.684749e-06
67901.0   -2.958251e-06
67902.0    4.473754e-06
67903.0    1.385007e-05
67904.0   -1.546003e-06
Length: 97391, dtype: float64, 'dollar': usd_volume_bucket
6.0         -3.768552e-09
9.0          1.193812e-06
12.0         9.282954e-07
15.0         3.479178e-07
19.0        -1.235237e-08
                 ...     
7178838.0   -6.677091e-07
7178846.0   -6.536866e-07
7178855.0   -7.235950e-07
7178863.0   -2.188992e-07
7178872.0    4.393840e-07
Length: 324545, dtype: float64, 'calendar': timeSt

{'20180725': {'tick': tick_bucket
  2.0       2.666084e-06
  3.0      -2.900328e-06
  4.0      -3.096897e-06
  5.0      -1.983080e-06
  6.0      -7.613414e-07
                ...     
  1618.0    1.331230e-05
  1619.0   -5.017481e-06
  1620.0    6.777258e-09
  1621.0    1.599355e-05
  1622.0    1.114729e-05
  Length: 3243, dtype: float64,
  'volume': volume_bucket
  2.0        3.801245e-07
  3.0        2.082894e-06
  4.0        1.570772e-06
  5.0       -1.872156e-07
  7.0        7.931050e-07
                 ...     
  67900.0   -2.684749e-06
  67901.0   -2.958251e-06
  67902.0    4.473754e-06
  67903.0    1.385007e-05
  67904.0   -1.546003e-06
  Length: 97391, dtype: float64,
  'dollar': usd_volume_bucket
  6.0         -3.768552e-09
  9.0          1.193812e-06
  12.0         9.282954e-07
  15.0         3.479178e-07
  19.0        -1.235237e-08
                   ...     
  7178838.0   -6.677091e-07
  7178846.0   -6.536866e-07
  7178855.0   -7.235950e-07
  7178863.0   -2.188992e-07
  71

In [17]:
df = testClass.load_and_format_data()[date]
input_dict = testClass.get_bars(df)
calendar_bar_df = (testClass.get_concat_data(testClass._bars_dict)['calendar_bars'])
tick_bar_df = testClass.get_concat_data(testClass._bars_dict)['tick_bars']
volume_bar_df = (testClass.get_concat_data(testClass._bars_dict)['volume_bars'])


2021-11-13 20:08:26,402 DataLoader  INFO Loading data for:20180725


/media/ak/WorkDrive/Data/TU1_Comdty_quotes/20180725.csv
/media/ak/WorkDrive/Data/TU1_Comdty_trades/20180725.csv


2021-11-13 20:08:26,899 DataLoader  INFO Loading data for:20180724


/media/ak/WorkDrive/Data/TU1_Comdty_quotes/20180724.csv
/media/ak/WorkDrive/Data/TU1_Comdty_trades/20180724.csv


2021-11-13 20:08:27,304 DataLoader  INFO Loading data for:20180517


...

2021-11-13 20:09:06,813 DataLoader  INFO Loading data for:20180504


/media/ak/WorkDrive/Data/TU1_Comdty_quotes/20180504.csv
/media/ak/WorkDrive/Data/TU1_Comdty_trades/20180504.csv


2021-11-13 20:09:07,255 DataLoader  INFO Loading data for:20180913


/media/ak/WorkDrive/Data/TU1_Comdty_quotes/20180913.csv
/media/ak/WorkDrive/Data/TU1_Comdty_trades/20180913.csv


2021-11-13 20:09:07,937 DataLoader  INFO Calculating calendar clock
2021-11-13 20:09:09,261 DataLoader  INFO Calculating volume clock
2021-11-13 20:09:40,246 DataLoader  INFO Calculating tick clock
2021-11-13 20:09:41,551 DataLoader  INFO Calculating usd volume clock


In [19]:
volume_bar_df

,BidSize_open,BidSize_high,BidSize_low,BidSize_close,QuoteTime_open,QuoteTime_high,QuoteTime_low,QuoteTime_close,type_x_open,type_x_high,type_x_low,type_x_close,BestBid_open,BestBid_high,BestBid_low,BestBid_close,TradeTime_open,TradeTime_high,TradeTime_low,TradeTime_close,AskSize_open,AskSize_high,AskSize_low,AskSize_close,BestAsk_open,BestAsk_high,BestAsk_low,BestAsk_close,TradeSize_open,TradeSize_high,TradeSize_low,TradeSize_close,TradePrice_open,TradePrice_high,TradePrice_low,TradePrice_close,TimeStamp_open,TimeStamp_high,TimeStamp_low,TimeStamp_close,TradeVolume_open,TradeVolume_high,TradeVolume_low,TradeVolume_close,total_traded_volume_open,total_traded_volume_high,total_traded_volume_low,total_traded_volume_close,milliSeconds_open,milliSeconds_high,milliSeconds_low,milliSeconds_close,dollar_traded_volume_open,dollar_traded_volume_high,dollar_traded_volume_low,dollar_traded_volume_close,timeStampIdx_open,timeStampIdx_high,timeStampIdx_low,timeStampIdx_close,micro_price_open,micro_price_high,micro_price_low,micro_price_close
volume_bucket,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.0,7325,7325,4088,4153,2018-07-25 00:02:48,2018-07-25 00:03:49,2018-07-25 00:02:48,2018-07-25 00:03:49,BID,BID,BID,BID,105.734375,105.734375,105.734375,105.734375,2018-07-25 00:02:48,2018-07-25 00:03:49,2018-07-25 00:02:48,2018-07-25 00:03:49,8624.0,10947.0,8624.0,10947.0,105.742188,105.742188,105.742188,105.742188,3.0,3.0,3.0,3.0,105.742188,105.742188,105.742188,105.742188,00:02:48,00:03:49,00:02:48,00:03:49,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,168000,229000,168000,229000,317.226562,317.226562,317.226562,317.226562,2018-07-25 00:02:05,2018-07-25 00:03:45,2018-07-25 00:02:05,2018-07-25 00:03:45,105.738599,105.740039,105.738599,105.740039
2.0,4088,4175,3920,4105,2018-07-25 00:03:49,2018-07-25 00:12:40,2018-07-25 00:03:49,2018-07-25 00:12:40,BID,BID,BID,BID,105.734375,105.734375,105.734375,105.734375,2018-07-25 00:03:49,2018-07-25 00:12:40,2018-07-25 00:03:49,2018-07-25 00:12:40,10947.0,11141.0,10926.0,11105.0,105.742188,105.742188,105.742188,105.742188,3.0,3.0,3.0,3.0,105.742188,105.742188,105.742188,105.742188,00:03:49,00:12:40,00:03:49,00:12:40,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,229000,760000,229000,760000,317.226562,317.226562,317.226562,317.226562,2018-07-25 00:03:45,2018-07-25 00:12:40,2018-07-25 00:03:45,2018-07-25 00:12:40,105.740063,105.740154,105.740035,105.740079
3.0,4152,4152,3086,3544,2018-07-25 00:13:02,2018-07-25 00:13:25,2018-07-25 00:13:02,2018-07-25 00:13:25,BID,BID,BID,BID,105.734375,105.734375,105.734375,105.734375,2018-07-25 00:13:02,2018-07-25 00:13:25,2018-07-25 00:13:02,2018-07-25 00:13:25,11106.0,12140.0,11106.0,11119.0,105.742188,105.742188,105.742188,105.742188,3.0,3.0,2.0,2.0,105.742188,105.742188,105.734375,105.734375,00:13:02,00:13:25,00:13:02,00:13:25,3.0,3.0,2.0,2.0,3.0,3.0,2.0,2.0,782000,805000,782000,805000,317.226562,317.226562,211.468750,211.468750,2018-07-25 00:13:02,2018-07-25 00:13:25,2018-07-25 00:13:02,2018-07-25 00:13:25,105.740062,105.740604,105.740062,105.740299
4.0,3479,3625,3086,3210,2018-07-25 00:13:25,2018-07-25 00:13:28,2018-07-25 00:13:25,2018-07-25 00:13:28,BID,BID,BID,BID,105.734375,105.734375,105.734375,105.734375,2018-07-25 00:13:25,2018-07-25 00:13:28,2018-07-25 00:13:25,2018-07-25 00:13:28,11119.0,11352.0,11119.0,11352.0,105.742188,105.742188,105.742188,105.742188,2.0,2.0,2.0,2.0,105.734375,105.734375,105.734375,105.734375,00:13:25,00:13:28,00:13:25,00:13:28,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,805000,808000,805000,808000,211.468750,211.468750,211.468750,211.468750,2018-07-25 00:13:25,2018-07-25 00:13:28,2018-07-25 00:13:25,2018-07-25 00:13:28,105.740326,105.740518,105.740267,105.740465
5.0,3245,3308,3241,3277,2018-07-25 00:13:28,2018-07-25 00:13:55,2018-07-25 00:13:28,2018-07-25 00:13:55,BID,BID,BID,BID,105.734375,105.734375,105.734375,105.734375,2018-07-25 00:13:28,2018-07-25 00:13:55,2018-07-25 00:13:28,2018-07-25 00:13:55,11352.0,11420.0,11352.0,11420.0,105.742188,105.742188,105.7

In [18]:


# # with Pool(number_of_workers) as p:
# with Pool(number_of_workers) as p:
#     print(p.map(produce_info_clock_files, intersectionDates))
